In [ ]:
# This will prompt you to select the file to upload
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
# This will prompt you to select the file to upload
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import wandb

# Step 1: Login to Weights & Biases
#wandb.login()  # If using API key, you can login with:
wandb.login(key="97c4966291215c34bb3c2aa45571bc0a38eb1387")

# Initialize the wandb run
wandb.init(
    project="NLPHW6",  # Replace with your project name
    name="distilbert-multi-label-run",  # Optional: Customize run name
    config={
        "learning_rate": 2e-5,
        "epochs": 20,
        "batch_size": 8,
        "weight_decay": 0.01,
    }
)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load train and test datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Define the emotion columns (target labels)
emotion_columns = ['anger', 'anticipation', 'disgust', 'fear', 'joy',
                   'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# Replace 'NONE' with 0 only in the test data (not needed for train data)
test_data[emotion_columns] = test_data[emotion_columns].replace('NONE', 0).astype(int)

# Initialize the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize tweets for DistilBERT
def encode_tweets(data):
    return tokenizer(
        list(data["Tweet"]),
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

# Encode train and test datasets
train_encodings = encode_tweets(train_data)
test_encodings = encode_tweets(test_data)

# Prepare labels for the training data
y_train = train_data[emotion_columns].values

# Extract input tensors from the encodings
train_input_ids = train_encodings["input_ids"]
train_attention_mask = train_encodings["attention_mask"]

# Ensure the number of samples matches with labels
print(f"Input IDs shape: {train_input_ids.shape}")
print(f"Labels shape: {y_train.shape}")

# Split the data into train and validation sets
train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    train_input_ids, train_attention_mask, y_train, test_size=0.2, random_state=42
)

# Convert to Hugging Face datasets
train_dataset = Dataset.from_dict({
    "input_ids": train_input_ids,
    "attention_mask": train_attention_mask,
    "labels": torch.tensor(train_labels, dtype=torch.float32)
})

val_dataset = Dataset.from_dict({
    "input_ids": val_input_ids,
    "attention_mask": val_attention_mask,
    "labels": torch.tensor(val_labels, dtype=torch.float32)
})

# Initialize the DistilBERT model for multi-label classification
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(emotion_columns),
    problem_type="multi_label_classification"
).to(device)

# Define metrics for evaluation
def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions)).numpy()
    preds_binary = (preds > 0.2).astype(int)  # Apply threshold of 0.2
    precision, recall, f1, _ = precision_recall_fscore_support(
        p.label_ids, preds_binary, average='macro'
    )

    # Log metrics to wandb
    wandb.log({"macro_f1": f1, "precision": precision, "recall": recall})

    return {"macro_f1": f1, "precision": precision, "recall": recall}

# Define training arguments with early stopping and wandb logging
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    logging_dir="./logs",
    logging_steps=10,
    greater_is_better=True,
    report_to="wandb"  # Enable wandb logging
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

# Evaluate the model on the validation set
val_results = trainer.evaluate()
print(f"Validation Results: {val_results}")

# Prepare the test dataset for predictions
test_dataset = Dataset.from_dict({
    "input_ids": test_encodings["input_ids"],
    "attention_mask": test_encodings["attention_mask"]
})

# Generate predictions on the test set
test_predictions = trainer.predict(test_dataset)
test_logits = test_predictions.predictions

# Apply a threshold of 0.2 to the predictions
test_preds_binary = (torch.sigmoid(torch.tensor(test_logits)) > 0.2).numpy().astype(int)

# Prepare submission file for Kaggle
submission_df = pd.DataFrame(test_preds_binary, columns=emotion_columns)
submission_df["ID"] = test_data["ID"]
submission_df = submission_df[['ID'] + emotion_columns]

# Save submission file
submission_df.to_csv("submission_distilbert.csv", index=False)
print("Submission file created: 'submission_distilbert.csv'")

# Finish the wandb run
wandb.finish()


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: maryannekiarie98 (maryannekiarie98-university-of-texas-at-dallas). Use `wandb login --relogin` to force relogin


<ipython-input-4-02802944be82>:37: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data[emotion_columns] = test_data[emotion_columns].replace('NONE', 0).astype(int)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Macro F1,Precision,Recall
1,0.338700,0.326357,0.505342,0.431173,0.621497
2,0.307700,0.313542,0.566704,0.507794,0.678717
3,0.228500,0.334547,0.572473,0.531607,0.666268
4,0.200700,0.333461,0.573795,0.528125,0.650320
5,0.158100,0.353225,0.571612,0.516131,0.654642
6,0.150600,0.376256,0.568873,0.511044,0.658981
7,0.150600,0.402055,0.557156,0.526406,0.603588


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Validation Results: {'eval_loss': 0.3334610164165497, 'eval_macro_f1': 0.5737953246827445, 'eval_precision': 0.5281246559966255, 'eval_recall': 0.6503204851040257, 'eval_runtime': 3.6868, 'eval_samples_per_second': 419.067, 'eval_steps_per_second': 52.621, 'epoch': 7.0}
Submission file created: 'submission_distilbert.csv'


eval/loss,▂▁▃▃▄▆█▃
eval/macro_f1,▁▇███▇▆█
eval/precision,▁▆██▇▇██
eval/recall,▃█▇▅▆▆▁▅
eval/runtime,▁▇▅▂▇█▄▂
eval/samples_per_second,█▂▄▇▂▁▅▇
eval/steps_per_second,█▂▄▇▂▁▅▇
macro_f1,▁▇███▇▆█
precision,▁▆██▇▇██
recall,▃█▇▅▆▆▁▅
test/runtime,▁
